## Model generator main file

In [ ]:
import numpy as np
import re
import unidecode
import os
import pickle
import sys

import matplotlib.pyplot as plt
%matplotlib inline

from keras                  import regularizers
from keras                  import Input, Model, Sequential
from keras.layers           import Bidirectional, Activation, TimeDistributed, Dense, RepeatVector, Embedding, Dropout, BatchNormalization
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.utils            import np_utils
from keras.callbacks        import EarlyStopping, TensorBoard, ModelCheckpoint

Import data and make folders

In [ ]:
data_path = 'data'
temp_path = data_path + '/temp'
backup_path = data_path + '/backup'
weights_path = data_path + '/weights'
train_history = data_path + '/train_history'

if not os.path.isdir(data_path):
    os.mkdir(data_path)
if not os.path.isdir(temp_path):
    os.mkdir(temp_path)
if not os.path.isdir(backup_path):
    os.mkdir(backup_path)
if not os.path.isdir(weights_path):
    os.mkdir(weights_path)
if not os.path.isdir(train_history):
    os.mkdir(train_history)

with open(data_path + '/joint_angle_data.pickle', 'rb') as file:
    joint_angle_data = pickle.load(file)
    file.close()
    
with open(data_path + '/power_data.pickle', 'rb') as file:
    power_data = pickle.load(file)
    file.close()

## Recurrent neural networks

Different model functions

In [ ]:
def setModelToRNN(self):
    self.model = Sequential()
    self.model.add(Bidirectional(SimpleRNN(64, return_sequences = True, kernel_initializer = 'random_uniform', bias_initializer = 'zero', input_shape = (self.X.shape[1], self.X.shape[2]))))
    #self.model.add(BatchNormalization())
    #self.model.add(Bidirectional(SimpleRNN(64, return_sequences = True, kernel_initializer = 'random_uniform', bias_initializer = 'zero', go_backwards = False)))
    #self.model.add(BatchNormalization())
    #self.model.add(Bidirectional(SimpleRNN(64, kernel_initializer = 'random_uniform', bias_initializer = 'zero')))
    self.model.add(Dropout(0.2))
    self.model.add(Dense(self.y.shape[1], activation='softmax'))
    self.model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

In [ ]:
def setModelToGRU(self):
    self.model = Sequential()
    self.model.add(Bidirectional(GRU(64, return_sequences = True, kernel_initializer = 'random_uniform', bias_initializer = 'zero', input_shape = (self.X.shape[1], self.X.shape[2]))))
    self.model.add(BatchNormalization())
    self.model.add(Bidirectional(GRU(64, return_sequences = True, kernel_initializer = 'random_uniform', bias_initializer = 'zero', go_backwards = False)))
    self.model.add(BatchNormalization())
    self.model.add(Bidirectional(GRU(64, kernel_initializer = 'random_uniform', bias_initializer = 'zero')))
    self.model.add(Dropout(0.2))
    self.model.add(BatchNormalization())
    self.model.add(Dense(self.y.shape[1], activation='softmax'))
    self.model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

In [ ]:
def setModelToLSTM(self):
    self.model = Sequential()
    self.model.add(Bidirectional(LSTM(64, return_sequences = True, kernel_initializer = 'random_uniform', bias_initializer = 'zero', input_shape = (self.X.shape[1], self.X.shape[2]))))
    self.model.add(BatchNormalization())
    self.model.add(Bidirectional(LSTM(64, return_sequences = True, kernel_initializer = 'random_uniform', bias_initializer = 'zero')))
    self.model.add(BatchNormalization())
    self.model.add(Bidirectional(LSTM(64, kernel_initializer = 'random_uniform', bias_initializer = 'zero')))
    self.model.add(Dropout(0.2))
    self.model.add(BatchNormalization())
    self.model.add(Dense(self.y.shape[1], activation='softmax'))
    self.model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

Other functions

In [ ]:
def setUpData(self, seq_length):

    self.data_history = []
    self.data_label = []
    self.seq_length = seq_length
    
    for n in range(0,int(len(joint_angle_data)/seq_length)):
        self.data_history.append(np.array(joint_angle_data[n*seq_length:(n+1)*seq_length]).T)
        


In [ ]:
data_history = []
data_label = []
seq_length = 10
    
for n in range(0,int(len(joint_angle_data)/seq_length)):
    data_history.append(np.array(joint_angle_data[n*seq_length:(n+1)*seq_length][1:len(joint_angle_data[0])]).T)
    #data_label.append()

#for n, TI_data in enumerate(joint_angle_data):
#        print(n)
#len(joint_angle_data)

print(len(data_history))
print(np.array(joint_angle_data[0:seq_length][1:len(joint_angle_data[0])+1]).T)

print(joint_angle_data[0:seq_length])

print(joint_angle_data[0])
print(joint_angle_data[0][1:len(joint_angle_data[0])])

In [ ]:
def trainModel(self, name):
    # File path for model
    filepath = weights_path + "/weights-" + name + "-{epoch:02d}-{loss:.4f}.hdf5"
    # Callbacks functions
    es = EarlyStopping(monitor='val_loss', patience=5, verbose=0)
    tb = TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32)
    mc = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='auto')
    # Train the model
    epochs = 20           #Maximum number of epochs to run
    batch_size = 128*2    #Size of training data batch
    Val_split = 0.1       #Procentage of training data to use as validation data
    history = self.model.fit(self.X, self.y, epochs=epochs, batch_size=batch_size, validation_split=Val_split, callbacks=[es, tb, mc])
    # Save the model
    filename = "model_" + name + ".hdf5"
    self.model.save_weights(weights_path + '/' + filename)
    # Save the history
    filename = "history_" + name + ".pickle"
    with open(train_history + '/' + filename, 'wb') as f:
        pickle.dump(history.history, f)
        f.close()

Model generator

In [ ]:
class Generator:

    set_up_data = setUpData
    train_model = trainModel
    
    set_model_to_RNN = setModelToRNN
    set_model_to_GRU = setModelToGRU
    set_model_to_LSTM = setModelToLSTM
    

## Training the network

In [ ]:
gen = Generator()
leng = 64
gen.set_up_data(leng)
gen.set_model_to_LSTM()
name = 'test_run'
gen.train_model(name = name)